# Wine and Food Pairing CLI Application

This notebook develops a command-line interface for wine and food pairing using the Textual library for rich terminal interfaces.

In [ ]:
# Wine and Food Pairing CLI Application
# This notebook creates a command-line interface for wine and food pairing using Textual

# First, let's install the required packages
!pip install textual rich

In [ ]:
from textual.app import App, ComposeResult
from textual.containers import Container, Horizontal, Vertical
from textual.widgets import Header, Footer, Input, Button, Static, Label, Select
from textual.reactive import reactive
from rich.text import Text
from rich.markdown import Markdown
import random
from typing import Dict, List, Tuple

# Wine and food pairing database
# Structure: { food_category: [(wine_type, explanation), ...] }
WINE_PAIRINGS = {
    "beef": [
        ("Cabernet Sauvignon", "The robust tannins in Cabernet Sauvignon complement the rich proteins in beef, creating a balanced taste experience. The wine's black fruit flavors enhance the savory aspects of the meat."),
        ("Malbec", "Malbec's plush fruit-forward profile and moderate tannins pair beautifully with the umami flavors in beef, especially with leaner cuts."),
        ("Syrah/Shiraz", "The peppery notes and full body of Syrah stand up to the strong flavors of beef, particularly with spiced or grilled preparations.")
    ],
    "chicken": [
        ("Chardonnay", "The buttery notes in Chardonnay complement the mild flavors of chicken, especially with creamy sauces. The wine's acidity cuts through richness while enhancing the meat's subtle flavors."),
        ("Pinot Noir", "For darker chicken meat or roasted preparations, Pinot Noir offers bright acidity and red fruit flavors that complement without overwhelming."),
        ("Sauvignon Blanc", "With lighter chicken dishes, particularly those with herbs or citrus, Sauvignon Blanc's crisp profile and herbal notes create a refreshing contrast.")
    ],
    "fish": [
        ("Pinot Grigio", "The light, crisp character of Pinot Grigio accentuates the delicate flavors of white fish while its acidity acts as a palate-cleanser."),
        ("Sauvignon Blanc", "The herbaceous qualities and high acidity of Sauvignon Blanc complement the natural oils in fish while adding brightness to the pairing."),
        ("Champagne", "The effervescence of Champagne cuts through the richness of fattier fish while its complexity enhances more luxurious seafood preparations.")
    ],
    "shellfish": [
        ("Chablis", "The flinty minerality and lean profile of Chablis mirrors the briny quality of shellfish, creating a harmonious flavor experience."),
        ("Muscadet", "Crisp, dry and with subtle salinity, Muscadet is almost tailor-made for oysters and other shellfish, enhancing their natural flavors."),
        ("Albariño", "The citrus notes and saline qualities of Albariño make it a natural companion to shellfish, particularly from coastal regions.")
    ],
    "pork": [
        ("Riesling", "The sweetness in Riesling balances beautifully with the natural sweetness of pork, while its acidity cuts through fat. Particularly good with glazed or fruit-accompanied pork."),
        ("Gewürztraminer", "The aromatic complexity and slight sweetness of Gewürztraminer complement pork's flavor, especially with preparations featuring warm spices."),
        ("Pinot Noir", "For roasted pork, the earthy qualities and medium body of Pinot Noir enhance the meat without overwhelming its more subtle characteristics.")
    ],
    "lamb": [
        ("Bordeaux Blend", "The structured tannins and complex fruit profile of Bordeaux blends stand up to lamb's distinctive flavor, creating a classic pairing."),
        ("Rioja", "The balanced oak influence and savory qualities of Rioja complement the gamey aspects of lamb, particularly with herb-crusted preparations."),
        ("Chianti", "The bright acidity and cherry notes of Chianti cut through lamb's richness while its earthy undertones match the meat's flavor intensity.")
    ],
    "game": [
        ("Barolo", "The powerful structure and distinctive flavor profile of Barolo can match the intense, complex flavors of game meats like venison."),
        ("Syrah", "The black pepper notes and bold fruit in Syrah complement the strong flavors in game meats, enhancing their savory qualities."),
        ("Châteauneuf-du-Pape", "The rich, spicy character of Châteauneuf-du-Pape pairs wonderfully with the distinctive flavors of game birds and meats.")
    ],
    "vegetarian": [
        ("Grüner Veltliner", "The bright, peppery qualities of Grüner Veltliner complement vegetable-forward dishes, particularly those featuring green vegetables."),
        ("Rosé", "The versatility of dry rosé makes it an excellent match for varied vegetarian flavors, offering fruit notes without overwhelming subtle vegetable components."),
        ("Beaujolais", "The light body and vibrant fruit in Beaujolais make it adaptable to many vegetarian dishes, particularly those with earthy components like mushrooms.")
    ],
    "spicy": [
        ("Gewürztraminer", "The slight sweetness and lychee notes of Gewürztraminer provide a counterbalance to spicy heat while its aromatics complement exotic spices."),
        ("Riesling", "Off-dry Riesling's combination of sweetness and acidity makes it perfect for spicy dishes, offering cooling relief from heat."),
        ("Moscato d'Asti", "The light sweetness and gentle bubbles in Moscato provide a refreshing contrast to spicy food, tempering the heat without competing with complex flavors.")
    ],
    "dessert": [
        ("Port", "The rich, concentrated sweetness of Port stands up to chocolate desserts while adding complementary dried fruit and spice notes."),
        ("Sauternes", "The honeyed richness of Sauternes pairs perfectly with fruit-based desserts, especially those featuring stone fruits or caramel."),
        ("Moscato d'Asti", "The light, sweet, floral character of Moscato complements delicate desserts and fruit-based treats with its gentle effervescence.")
    ],
    "cheese": [
        ("Port", "The sweetness and complexity of Port create a classic contrast with blue cheeses, while complementing the umami qualities."),
        ("Champagne", "The high acidity and bubbles of Champagne cut through the richness of soft cheeses like Brie,